<a href="https://colab.research.google.com/github/mateusjunges/music-gender-detection/blob/master/music-gender-detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import cv2
import numpy as np
import pandas as pd
import re
import math
from skimage import feature
from skimage.feature import hog
from imutils import paths
from google.colab.patches import cv2_imshow
import os
import librosa
import matplotlib.pyplot as plt
%matplotlib inline
import csv

from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Input

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

In [0]:
from keras import models
from keras import layers

In [0]:
# mount the google drive:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Path to the songs:

base_path        = "/content/drive/My Drive"
workspace_folder = base_path + '/UEPG/eng-comp/4-ano/pattern-recognition/trabalho-musicas'
main_folder_path = base_path + '/GTZAN' # The path to the main folder
genders_path     = main_folder_path + '/genres'




In [0]:
# Creating a dataset:

# Generating headers for our CSV file.
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [0]:
# Calculating features:
data_csv = workspace_folder + '/data.csv'
file = open(data_csv, 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'
genres = genres.split()

print(genders_path)
for gender in genres:
    for filename in os.listdir(f'{genders_path}/{gender}'):
        songname = f'{genders_path}/{gender}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {gender}'
        file = open(data_csv, 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

/content/drive/My Drive/GTZAN/genres


In [0]:
# Processing the generated CSV file:
data = pd.read_csv(data_csv)
data.head()

In [0]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1) # The filename column is not required
data.head()

In [0]:
# Encode genres into integers

genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

In [0]:
# Normalizing the dataset:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float)) # x is calculated by removing the mean and dividing by the variance


In [0]:
# Split the dataset into training and test:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% train, 30% test


In [0]:
# Creating the model, training and testing:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# There are 4 layers in our network. First one is an input layer therefore, input size has to be given. 
# Then there are 2 hidden layers and the last layer is an output layer. 
# The value inside dense represents the dimension of an output space.
# So the 1st layer has 256 neurons so the dimension of it’s output space is 256.
# Such that the output layer has 10 neurons as we are classifying into 10 genres.


In [0]:
# Learning process of the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# Training the model:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)


In [0]:
# Evaluating the model:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('Test Accuracy: ', test_acc)

300/300 [==============================] - 0s 60us/step
Test Accuracy:  0.6299999996026358


In [0]:
# Predictions
predictions = model.predict(X_test)
np.argmax(predictions[0])

In [0]:
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network  import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Creating all classifiers:
svm_classifier = SVC(gamma='scale')

# parameters for SVM
parameters = [
  {'C': [0.1, 0.5, 1, 10, 100, 500, 1000], 'kernel': ['poly']},
  {'C': [0.1, 0.5, 1, 10, 100, 500, 1000], 'gamma': [0.1, 0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
]

svm = GridSearchCV(svm, parameters, scoring = 'accuracy', cv=10, iid=False)
logistic_regression = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
nb = GaussianNB(var_smoothing=1e-09)

rng = np.random.RandomState(123)
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(100), activation='logistic', batch_size=100, max_iter=2000, learning_rate_init=0.1, momentum=0.2, tol=1e-10, random_state=rng)

methods = [svm_classifier, logistic_regression, nb, mlp]
titles = ["SVM", "Logistic Regression", "Naive Bayes", "MLP"]

In [0]:
scores = []
cross_validation_results = []

folds = 10

for method, name in zip(methods, titles):
  method.fit(X_train, y_train)
  scores.append(method.score(X_test, y_test))
  method_score = method.score(X_test, y_test)
  print("Classification accuracy for {} = {}"
        .format(name, method_score))
  # Using cross validation:
  result = model_selection.cross_val_score(method, X_train, y_train, cv=folds)
  cross_validation_results.append(result)
  print("Result using cross validation for {} = {}"
        .format(name, result.mean()))

  
# Classification accuracy for SVM = 0.6166666666666667
# Result using cross validation for SVM = 0.6503404406977297
# Classification accuracy for Logistic Regression = 0.61
# Result using cross validation for Logistic Regression = 0.5956469313731987
# Classification accuracy for Naive Bayes = 0.43333333333333335
# Result using cross validation for Naive Bayes = 0.4234869954804529
# Classification accuracy for MLP = 0.6833333333333333
# Result using cross validation for MLP = 0.6615183732887056